In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as so

# Flow Rate and  Friction Losses

## Energy Balance 

From our energy balance, we obtain the following equation that relates $f_f$ and $v$

$$ 0 = -\frac{\Delta P}{\rho} + \frac{2 L}{D}f_f v^2 \implies{v = \sqrt{\frac{D \Delta P}{2 L \rho f_f}}} $$

## Colebrook Equation

Implicit expression, page 190 of _Fundamentals of Momentum, Heat and Mass Transfer_. Most rigorous expression for friction factor. 

$$ \frac{1}{\sqrt{f_f}} = 4 \log_{10}\left(\frac{D}{e} \right) + 2.28 - 4 \log_{10}\left(4.67 \frac{D / e}{\text{Re} \sqrt{f_f}} + 1\right)$$

Can solve for $f_f$ by finding roots of the following function via Newton's method

$$ h(f_f) = \frac{-1}{\sqrt{f_f}} + 4 \log_{10}\left(\frac{D}{e} \right) + 2.28 - 4 \log_{10}\left(4.67 \frac{D / e}{\text{Re} \sqrt{f_f}} + 1\right) $$

This function's derivative is given by 

$$ h^\prime(f_f) = \frac{f_f^{-3/2}}{2}\left[1 + \left(\frac{1.737178}{4.67\frac{D / e}{\text{Re} \sqrt{f_f}} + 1}\right)\left(\frac{4.67 D / e}{\text{Re}}\right)\right]$$

And the iterative steps of Newton's method are expressed by 

$$ x_{n+1} = x_n - \frac{h(x_n)}{h^\prime(x_n)}$$

In [2]:
def newton_raphson(f,fprime,guess):
    """
    f = function f(x)
    fprime = function f'(x)
    guess = initial guess for roots of f
    """
    
    tol = 1e-8
    x1 = guess

    while True:
        x2 = x1 - f(x1)/fprime(x1)
        
        if abs((x2 - x1)/x1) <= tol:
            break
        
        x1 = x2
        
    return(x2)

## Haaland's Equation 

Explicit expression, page 191 of _Fundamentals of Momentum, Heat and Mass Transfer_. Explicit Approximation of the Colebrook equation—we will use it to supply an initial guess to Newton's method. 

$$ \frac{1}{\sqrt{f_f}} = -3.6 \log_{10}\left(\frac{6.9}{\text{Re}} + \left(\frac{e}{3.7 D}\right)^{10/9} \right) $$

In [3]:
def pipe_friction(P,rho,mu,L,D,e,f_in):
    """
    P = Pressure drop [Pa]
    rho = Fluid density [kg/m^3]
    mu = Dynamic viscosity [Pa s]
    L = Pipe length [m]
    D = Pipe diameter [m]
    e = Absolute roughness [m]
    f_in = Initial guess for friction factor [ ]
    
    This function solves for the flow of fluid in a 
    length of pipe driven by pressure.
    """
    
    tol = 1e-8
    
    g = -9.81 # [m/s^2] acceleration due to gravity
    
    f1 = f_in
    
    while True:
        
        v = np.sqrt(D*P/(2*L*rho*f1))
     
        Re = rho*v*D/mu
    
        def h(f): # Colebrook Equation 
        
            term1 = -1/f**0.5
            term2 = 4*np.log10(D/e) + 2.28
            term3 = -4*np.log10(4.67*(D/e)/(Re*f**0.5) + 1)
        
            return(term1 + term2 + term3)
    
        def hprime(f):
        
            term1 = 4.67*D/(Re*e)
            term2 = 1 + 1.737178/(term1/f**0.5 + 1)*term1
            term3 = f**-1.5/2*term2
        
            return(term3)
    
        # Haaland Equation 
        f_guess = (-3.6*np.log10(6.9/Re + (e/(3.7*D))**(10/9)))**-2
        
        # Newton's Method
        f2 = newton_raphson(h,hprime,f_guess)
        
        if (f2-f1)/f1 <= tol:
            v = np.sqrt(D*P/(2*L*rho*f2))
            Re = v*rho*D/mu
            break 
            
        f1 = f2
        
    return(v,Re,f2)

In [4]:
def pipe_head_loss(rho,mu,v,D,e,L):
    """
    rho = Fluid density [kg/m^3]
    mu = Dynamic viscosity [Pa s]
    v = Fluid velocity [m/s]
    D = Pipe diameter [m]
    e = absolute roughness [m]
    L = Pipe length [m]
    
    This function solves for the head loss due 
    to friction when the flow rate is known.
    """
    
    tol = 1e-8
    g = 9.81 # m/s^2
    
    Re = rho*v*D/mu
    
    def h(f): # Colebrook Equation 
        
        term1 = -1/f**0.5
        term2 = 4*np.log10(D/e) + 2.28
        term3 = -4*np.log10(4.67*(D/e)/(Re*f**0.5) + 1)
        
        return(term1 + term2 + term3)
    
    def hprime(f):
        
        term1 = 4.67*D/(Re*e)
        term2 = 1 + 1.737178/(term1/f**0.5 + 1)*term1
        term3 = f**-1.5/2*term2
        
        return(term3)
        
    # Haaland Equation 
    f_guess = (-3.6*np.log10(6.9/Re + (e/(3.7*D))**(10/9)))**-2
    
    # Newton's Method
    f = newton_raphson(h,hprime,f_guess)
    
    head_loss = 2*f*L/D*v**2/g
    
    return(head_loss,Re,f)

# Physical Property Data 

Physical properties are to some degree temperature dependent, and should be evaluated at the arithmetic mean temperature of the fluid in our heat exchanger. Properties of water and 10% by weight aqueous solutions of glucose are available as either tabulated data (see references 1,9, and 10) or as polynomial functions of $T$ (see reference 11). The following functions will make it easy for our program to update these values with each iteration. 


In [5]:
def interpolate(x,x_data,y_data):
    """
    x = Value of independent variable 
    x_data = Array of tabulated values of x
    y_data = Array of corresponding values of y
    
    This function interpolates the value of y(x) from tabulated data 
    """
    
    bounds = 0
    last_point = 0
    
    for i in range(0,len(x_data)):
        if x_data[i] <= x:
            start = i
            bounds = 1
            if i == len(x_data) - 1:
                last_point = 1
            
        if x_data[i] > x and bounds == 1:
            end = i
            break
    
    if last_point != 1:
        x1 = x_data[start]
        y1 = y_data[start]
        x2 = x_data[end]
        y2 = y_data[end]
    
        y = (y2 - y1)/(x2 - x1)*(x - x1) + y1
    else:
        y = y_data[-1]
    
    return(y)

In [6]:
def water_physical_properties(T):
    """
    T = Temperature [C]
    
    This function finds the physical properties of water 
    at different temperatures
    """
    
    # Tabulated data 
    T_data = [0,20,40,60,80,100] # Temperature [C]
    mu_data = [1794e-6,993e-6,538e-6,472e-6,352e-6,278e-6] # Dynamic viscosity [Pa s]
    Cp_data = [4226,4182,4175,4181,4194,4211] # Isobaric heat capacity [J / kg K]
    k_data = [0.558,0.597,0.633,0.658,0.673,0.682] # Thermal conductivity [W / m K]
    
    rho = 997 # density [kg / m^3]
    mu = interpolate(T,T_data,mu_data)
    Cp = interpolate(T,T_data,Cp_data)
    k = interpolate(T,T_data,k_data)
    
    return(rho,mu,Cp,k)

In [7]:
def wort_physical_properties(T):
    """
    T = Temperature [C]
    
    This function finds the physical properties of wort at different temperatures
    Approximate wort as 10 w% aqueous solution of glucose 
    """
    
    # Density - Reference 9
    rho = 1079.7 # [kg/m^3]
    
    # Viscosity - Reference 10
    mu_T_data = [0,5,10,15,20,25,35,40,45,50,55,60,65,70,80,85,95]
    mu_data = [1.95,1.68,1.77,1.39,1.24,1.07,0.98,0.81,0.71,0.72,
               0.57,0.51,0.43,0.46,0.31,0.28,0.21] # [mPa s]
    mu = interpolate(T,mu_T_data,mu_data)/1000 # [Pa s]
    
    # Heat capacity - Reference 11
    # Heat capacity data for binary solutions fit to following model 
    a1 = 4.15263; b1 = -0.03271e-4; c1 = 1.994e-8
    a2 = -0.00107; b2 = 1.999e-8; c2 = -0.014e-12
    a3 = 0.174e-4; b3 = 0.013e-8; c3 = 0.0013e-13
    S = 0.1*rho # S is %w/v in g/mL --> S = weight percent * density
    term1 = (a1 + b1*S + c1*S**2)
    term2 = (a2 + b2*S + c2*S**2)*T
    term3 = (a3 + b3*S + c3*S**2)*T**2
    Cp = (term1 + term2 + term3)*1000 # [J / kg K]
    
    # Thermal conductivity - couldn't find good source, will approximate as water 
    k_T_data = [0,20,40,60,80,100] # Temperature [C]
    k_data = [0.558,0.597,0.633,0.658,0.673,0.682] # Thermal conductivity [W / m K]
    k = interpolate(T,k_T_data,k_data)
    
    return(rho,mu,Cp,k)

# Flow Character for Water and Wort Streams

These functions will allow us to solve for the flow character of each stream for a given length of pipe, and incorporate the aspects of our heat exchanger geometry that we chose to fix. All calculations were done in the metric system to avoid mistakes, although end results are sometimes displayed in AES for ease of comprehension. 

In [8]:
def water_flow(rho,mu,L,P):
    """
    rho = density [kg / m^3]
    mu = viscosity [Pa s]
    L = length pipe [m]
    P = Pressure drop [Pa]
    """
    Di = 0.0127 # [m] Inner pipe outer diameter - (3/8 in size has 0.5 in OD)
    Do = 0.0254 # [m] Outer pipe inner diameter - (1 in size has 1 in ID)
    Ri = Di/2
    Ro = Do/2
    Acs = np.pi*(Ro**2 - Ri**2) # [m^2] Cross section area 
    Wetted_Perim = np.pi*(Di + Do) # [m] Wetted Perimeter
    DH = Acs/Wetted_Perim # [m] Hydraulic Diameter 
    D = 4*DH # [m] Equivalent Diameter - 0.625 in when Ri = 3/8 in, Ro = 1 in
    eD = 0.00004 # [] e/D Relative roughness, Figure 13.2, p.193 (drawn tubing)
    e = eD*D
    f_guess = 0.001

    v,Re,f = pipe_friction(P,rho,mu,L,D,e,f_guess)

    V = Acs*v # [m^3/s] volumetric flow rate
    m = V*rho # [kg / s] mass flow rate 

    return(m,v,Re,f)

In [9]:
def wort_flow(rho,mu,L,V):
    """
    rho = density [kg / m^3]
    mu = viscosity [Pa s]
    L = length pipe [m]
    V = Volumetric flow rate [m^3/s]
    """
    g = 9.81 # [m/s^2]
    D = 0.0102108 # [m] Pipe inner diameter - 0.402 in 
    R = D/2 # [m] Pipe radius
    Acs = np.pi*R**2 # [m] Cross sectional area
    eD = 0.00004 # [] e/D Relative roughness, Figure 13.2, p.193 (assumes drawn tubing)
    e = eD*D
    f_guess = 0.001
    
    m = V*rho # [kg / s] mass flow rate 
    v = V/Acs # [m/s] Average velocity
    
    hL,Re,f = pipe_head_loss(rho,mu,v,D,e,L)
    
    W_pump = m*g*hL # [Watts] Power required
    
    return(m,v,Re,f,hL,W_pump)

# Heat transfer 

The overall heat transfer coefficient in our system is given by 

$$ UA = \left[\frac{1}{A_i h_i} + \frac{\ln\left(r_o/r_i\right)}{2\pi k L} + \frac{1}{A_o h_o}\right]^{-1} $$

The convective heat transfer coefficients, $h_i$ and $h_o$, can be estimated from the Dittus-Boelter Correlation, which relates the following dimensionless groups

$$ \text{Re} = \frac{\rho v D}{\mu} \hspace{1cm} \text{Pr} = \frac{c_p \mu}{k} \hspace{1cm} \text{Nu} = \frac{hD}{k}$$

For turbulent flow, 

$$ \text{Nu} = 0.023\, \text{Re}^{0.8}\, \text{Pr}^n$$

Where $n$ is 0.3 for a fluid being cooled, and 0.4 for a fluid being heated. Once $UA$ is known, the NTU method can be applied to determine the heat exchanger's effectiveness

$$ \epsilon = \frac{1 - \exp\left[-\text{NTU}\left(1 - \frac{C_{min}}{C_{max}}\right)\right]}
{1 - \left(\frac{C_{min}}{C_{max}}\right)\exp\left[-\text{NTU}\left(1 - \frac{C_{min}}{C_{max}}\right)\right]}
$$

Where NTU is the ratio $UA/C_{min}$. Once the effectiveness has been determined, it is possible to solve for the outlet flow rate using equation (9).

In [10]:
def UA_overall(k,L,ri,ro,Ai,hi,Ao,ho):
    """
    k = Thermal conductivity [W/(m K)]
    L = Heat exchanger length [m]
    ri = Pipe inner radius [m]
    ro = Pipe outer radius [m]
    hi = Inner convective heat transfer coefficient [W/(m^2 K)]
    ho = Outer convective heat transfer coefficient [W/(m^2 K)]
    """
    
    Ri_conv = 1/(Ai*hi)
    R_cond = np.log(ro/ri)/(2*np.pi*k*L)
    Ro_conv = 1/(Ao*ho)
    UA = 1/(Ri_conv + R_cond + Ro_conv)
    
    return(UA)

In [11]:
def Effectiveness(UA,C_min,C_max):
    """
    UA = Overall heat transfer coefficient * Area [W/K]
    C_min = Minimum heat capacity rate [W/K]
    C_max = Maximum heat capacity rate [W/K]
    """
    
    NTU = UA/C_min
    numerator = 1 - np.exp(-NTU*(1 - C_min/C_max))
    denominator = 1 - (C_min/C_max)*np.exp(-NTU*(1 - C_min/C_max))
    E = numerator/denominator
    return(E)
    

In [12]:
def Dittus_Boelter(Re,Pr,side):
    """
    Re = Reynolds number []
    Pr = Prandtl number []
    side = string, specifies if fluid is being cooled or heated 
    """
    
    if side == 'h':
        n = 0.3 # Hot side --> fluid being cooled
    else:
        n = 0.4 # Cold side --> fluid being heated 
    Nu = 0.023*Re**0.8*Pr**n
    return(Nu)

# Bringing it Together 

We'll put together the functions we've built up into one large function that will take our process data as an input, and return the dependent variables in our system as outputs. Due to the temperature dependence of physical properties, iteration will occur until the solution has converged.  

In [13]:
def Heat_Exchanger(Tc_in,Th_in,P_c,V_h,L):
    """
    Tc_in = cold side inlet temp [C]
    Th_in = hot side inlet temp [C]
    P_c = cold side pressure drop [Pa]
    V_h = hot side volumetric flow [m^3/s]
    L = heat exchanger length [m]
    """
    
    # System Geometry 
    D_ii = 0.0102108 # [m] Inner Pipe ID - 0.402 in
    D_io = 0.0127 # [m] Inner Pipe OD - 0.5 in 
    D_oi = 0.0254 # [m] Outer pipe inner diameter - (1 in size has 1 in ID)
    R_ii = D_ii/2 
    R_io = D_io/2
    Ai = 2*np.pi*R_ii*L # [m^2] Inner surface area  
    Ao = 2*np.pi*R_io*L # [m^2] Outer surface area 
    k_pipe = 386 # [W/(m K)] thermal conductivity of copper 
    
    # Initially evaluate physical properties at inlet temperatures
    
    Tc_out = Tc_in
    Th_out = Th_in
    
    # Convergence criteria
    tol = 1e-10
    
    while True:
        
        # Arithmetic mean temperatures
        Tc_avg = (Tc_in + Tc_out)/2
        Th_avg = (Th_in + Th_out)/2
        
        Tc_out_old = Tc_out
        Th_out_old = Th_out

        # Get physical properties 
        rho_c,mu_c,Cp_c,k_c = water_physical_properties(Tc_avg)
        rho_h,mu_h,Cp_h,k_h = wort_physical_properties(Th_avg)
        
        # Determine flow properties within pipes
        m_c,v_c,Re_c,f_c = water_flow(rho_c,mu_c,L,P_c)
        m_h,v_h,Re_h,f_h,hL,W_pump = wort_flow(rho_h,mu_h,L,V_h)
        
        # Prandtl number 
        Pr_c = Cp_c*mu_c/k_c
        Pr_h = Cp_h*mu_h/k_h
        
        # Nusselt number 
        Nu_c = Dittus_Boelter(Re_c,Pr_c,'c')
        Nu_h = Dittus_Boelter(Re_c,Pr_c,'h')
        
        # Convective heat transfer coefficients
        D_eq = D_oi - D_io # [m] Equivalent diameter for annalus (water flow)
        ho = Nu_c*k_c/D_eq
        hi = Nu_h*k_h/D_ii
    
        # Overall heat transfer coefficient 
        UA = UA_overall(k_pipe,L,R_ii,R_io,Ai,hi,Ao,ho)
        U = UA/Ao
        
        # Use NTU method to estimate rate of heat transfer 
        C_c = m_c*Cp_c
        C_h = m_h*Cp_h
    
        if C_h < C_c:
            C_min = C_h
            C_max = C_c
        else:
            C_min = C_c
            C_max = C_h
        
        # Heat exchanger effectiveness
        E = Effectiveness(UA,C_min,C_max)
    
        # Heat transferred 
        q = E*C_min*(Th_in - Tc_in)
    
        # Outlet temperatures 
        Tc_out = q/C_c + Tc_in
        Th_out = Th_in - q/C_h
        
        change_c = np.abs(Tc_out - Tc_out_old)/Tc_out
        change_h = np.abs(Th_out - Th_out_old)/Th_out
        
        if change_c < tol and change_h < tol:
            break
            
    cold_properties = [rho_c,mu_c,Cp_c,k_c]
    hot_properties = [rho_h,mu_h,Cp_h,k_h]
    cold_flow = [m_c,v_c,f_c,Re_c,Pr_c,Nu_c,Tc_out]
    hot_flow = [m_h,v_h,f_h,Re_h,Pr_h,Nu_h,Th_out]
    heat_exchange = [E,U,q]
    pump_size = [hL,W_pump]
    
    return(cold_properties,hot_properties,cold_flow,hot_flow,heat_exchange,pump_size)